In [1]:
import onshape as brepio
from visualize import plot_assembly, plot_mate, emptyplot
import os
import pandas as ps
import ipywidgets as widgets
from IPython.display import clear_output

In [2]:
df_name = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5'
df_name_part = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5_segmentation.h5'
assembly_df = ps.read_hdf(df_name,'assembly')
part_df = ps.read_hdf(df_name_part,'part')
mate_df = ps.read_hdf(df_name,'mate')

In [3]:
path_to_index = ps.Series(dict((v,k) for k,v in assembly_df['AssemblyPath'].iteritems()))

In [4]:
has_geometry = part_df.groupby('Assembly')['HasGeometry'].agg(all)
assembly_df['HasAllGeometry'] = has_geometry

In [5]:
part_df.set_index('Assembly', inplace=True)

In [6]:
datapath = '/projects/grail/benjones/cadlab'

In [7]:
loader = brepio.Loader(datapath)

In [16]:
mate_types = ['PIN_SLOT', 'BALL', 'PARALLEL', 'SLIDER', 'REVOLUTE', 'CYLINDRICAL', 'PLANAR', 'FASTENED']
@mp.interact(sample="7492")
def display_sample(sample):
    try:
        sample = int(sample)
    except ValueError:
        sample = path_to_index[sample]
    assemblypath = assembly_df.loc[sample, "AssemblyPath"]
    did, mv, eid = assemblypath.split('_')
    print(f'https://cad.onshape.com/documents/{did}/w/8a6b7ec29c9e253f04b1a1fb/m/{mv}/e/{eid}')
    print(assemblypath, f' ({sample})')
    try:
        geo, mates = loader.load_flattened(assemblypath + '.json', skipInvalid=True)
    except FileNotFoundError as e:
        print(f'File not found: {e}')
        return
    assert(list(geo) == list(part_df.loc[sample,'PartOccurrenceID']))
    mate_counts = dict()
    for mate in mates:
        if len(mate.matedEntities) == 2:
            if mate.type not in mate_counts:
                mate_counts[mate.type] = 0
            mate_counts[mate.type] += 1
    num_connected = assembly_df.loc[sample, "ConnectedComponents"]
    num_rigid = assembly_df.loc[sample, "RigidPieces"]
    if num_connected > 1:
        print('warning:',num_connected,'connected components')
    print('rigid pieces:',num_rigid)
    print('total parts:',len(geo))
    print(f'mates: {len(mates)}: ',mate_counts)

    #choices = [(f'mate {i} ({mates[i].type}) ({mates[i].matedEntities[0][0]}, {mates[i].matedEntities[1][0]})',i) for i in range(len(mates)) if len(mates[i].matedEntities) == 2]
    choices = [(f'mate {i} ({mates[i].type}) ({mates[i].name})',i) for i in range(len(mates)) if len(mates[i].matedEntities) == 2]
    choices = [('fullAssembly', -1)] + choices
    p = emptyplot()
    badOccs = [k for k in geo if geo[k][1] is None or geo[k][1].V.shape[0] == 0]
    if len(badOccs) > 0:
        print(f'warning: {len(badOccs)} invalid parts!')
    #for o in badOccs:
    #    geo.pop(o)
    @mp.interact(mate=choices, wireframe=False, show_parts=True, use_rigid_labels=True)
    def ff(mate, wireframe, show_parts, use_rigid_labels):
        if mate == -1:
            print('displaying full assembly')
            if use_rigid_labels:
                rigid_labels = list(part_df.loc[sample, 'RigidComponentID'])
            else:
                rigid_labels = None
            plot_assembly(geo, mates, p=p, wireframe=wireframe, show_parts=show_parts, rigid_labels = rigid_labels)
            #print('num mates:',len(mates))
        elif len(mates[mate].matedEntities) == 2:
            me = mates[mate].matedEntities
            print('mated parts:',me[0][0],me[1][0])
            if me[0][0] in badOccs or me[1][0] in badOccs:
                print('invalid parts in mate')
                return
            plot_mate(geo, mates[mate], p=p, wireframe=wireframe)
                
        else:
            print(f'nonstandard mate with {len(me)} entities')
    p

interactive(children=(Text(value='7492', description='sample'), Output()), _dom_classes=('widget-interact',))

In [19]:
assembly_df.index[-1]

125132